In [1]:
// Cell 1: Import necessary dependencies
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.13.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-serialization-json:1.5.1")


In [2]:
// Cell 2: Import statements
import com.fasterxml.jackson.annotation.JsonFormat
import com.fasterxml.jackson.annotation.JsonProperty
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import com.fasterxml.jackson.module.kotlin.readValue
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.lookup.PlanningId
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.variable.PlanningVariable
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import ai.timefold.solver.core.api.score.stream.ConstraintCollectors
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.api.solver.Solver
import java.io.File
import java.time.Duration
import java.time.LocalTime


In [3]:
// Cell 3: Define Location class
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    fun distanceTo(other: Location): Double {
        val earthRadius = 6371e3 // Earth radius in meters
        val lat1 = Math.toRadians(latitude)
        val lat2 = Math.toRadians(other.latitude)
        val deltaLat = Math.toRadians(other.latitude - latitude)
        val deltaLon = Math.toRadians(other.longitude - longitude)

        val a = Math.sin(deltaLat / 2) * Math.sin(deltaLat / 2) +
                Math.cos(lat1) * Math.cos(lat2) *
                Math.sin(deltaLon / 2) * Math.sin(deltaLon / 2)
        val c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1 - a))

        val distance = earthRadius * c // Distance in meters
        return distance
    }
}


In [4]:
// Cell 4: Define Truck and Client Classes

// Define Truck class as a problem fact
data class Truck(
    @JsonProperty("Truck_ID")
    val truckId: String,

    @JsonProperty("Truck_Type")
    val truckType: String, // e.g., "Multilift 4 essieux", "Multilift 4 essieux Tridem", "Camion-grue 4 essieux"

    @JsonProperty("Capacity_Kg")
    val capacityKg: Double,

    @JsonProperty("Equipment")
    val equipment: String, // e.g., "Benne 10 m3", "Benne 20 m3", "Pont 13 m3", "Thermosilo"

    @JsonProperty("Can_Access_Difficult_Locations")
    val canAccessDifficultLocations: Boolean,

    @JsonProperty("Home_Location")
    val homeLocation: Location // Warehouse location
) {
    override fun toString(): String = truckId
}

// Define Client class as a planning entity
@PlanningEntity
data class Client(
    @JsonProperty("Client_ID")
    val clientId: String,

    @JsonProperty("Need")
    val need: String, // "Waste", "Enrobé", "Grue"

    @JsonProperty("Volume_t")
    val volumeT: Double,

    @JsonProperty("Volume_kg")
    val volumeKg: Double,

    @JsonProperty("Access_Difficulty")
    val accessDifficulty: String, // "Yes" or "No"

    @JsonProperty("Time_Window_Label")
    val timeWindowLabel: String, // "Morning", "Afternoon", "All Day"

    @JsonProperty("Time_Window_Start")
    val timeWindowStart: String, // "08:00", "13:00", etc.

    @JsonProperty("Time_Window_End")
    val timeWindowEnd: String,

    @JsonProperty("Latitude")
    val latitude: Double,

    @JsonProperty("Longitude")
    val longitude: Double
) {
    val location: Location = Location(latitude, longitude)

    // Convert "Yes"/"No" to Boolean
    val hasAccessDifficulty: Boolean
        get() = accessDifficulty.equals("Yes", ignoreCase = true)

    // Time windows as LocalTime
    val timeWindowStartTime: LocalTime
        get() = LocalTime.parse(timeWindowStart, DateTimeFormatter.ofPattern("HH:mm"))

    val timeWindowEndTime: LocalTime
        get() = LocalTime.parse(timeWindowEnd, DateTimeFormatter.ofPattern("HH:mm"))

    // Planning variable: Assign a Truck to this Client
    @PlanningVariable(valueRangeProviderRefs = ["truckRange"], nullable = false)
    var truck: Truck? = null

    // No-arg constructor for Timefold
    constructor() : this(
        clientId = "",
        need = "",
        volumeT = 0.0,
        volumeKg = 0.0,
        accessDifficulty = "No",
        timeWindowLabel = "",
        timeWindowStart = "00:00",
        timeWindowEnd = "00:00",
        latitude = 0.0,
        longitude = 0.0
    )

    override fun toString(): String = clientId
}


Line_4.jupyter.kts (67:50 - 67) Unresolved reference: DateTimeFormatter
Line_4.jupyter.kts (70:48 - 65) Unresolved reference: DateTimeFormatter